### CONSIGNA

Se  tiene  información  estadística  de  la  temporada  regular  de  todos  los jugadores   de   la   NBA   en   un   RDD   de   tuplas   con   el   siguiente   formato:

<span style="color:darkred"><em>
(id_jugador, nombre, promedio_puntos, promedio_asistencias, promedio_robos, promedio_bloqueos,  promedio_rebotes,  promedio_faltas). 
</em></span> 

Un  analista  de  la cadena ESPN  está  trabajando  con  un  RDD que corresponde  a  la primera  ronda  de playoffs y que tiene el siguiente formato: 

<span style="color:darkred"><em>
(id_jugador, id_partido, timestamp, cantidad_puntos, cantidad_rebotes, cantidad_bloqueos, cantidad_robos, cantidad_asistencias,   cantidad_faltas).
</em></span>
  

<br />


En   base   a estos   RDDs   se   quiere  programar  en  PySpark  un  programa  que  genere  un  RDD  con  los  nombres  (sin duplicados)  de  los  jugadores  que  lograron  en  algún  partido  de  playoffs  una  cantidad de asistencias mayor a su promedio histórico (el de la temporada regular).

### Llamaremos:

rdd_po: RDD con los datos de los playoffs. <br />
rdd_tr: RDD con los datos de temporada regular.

### Codigo:

In [4]:
# Por si hay mas de un contexto de PySpark corriendo (por ejemplo, otro Notebook), esto para utilizar el mismo.
sc = SparkContext.getOrCreate()

In [ ]:
#Dejo al RDD de playoffs solo con tuplas de formato ((id_jugador, id_partido), cantidad_asistencias) 
rdd_po = rdd_po.map(lambda x: ( (x[0],x[1]), x[7] ))

#Dejo al RDD de temporadas regulares solo con tuplas de formato (id_jugador, [nombre, promedio_asistencias]) 
rdd_tr = rdd_tr.map(lambda x: ( x[0], [x[1], x[3]] ))

In [ ]:
# En este paso, se toma el RDD de playoffs.
# Primero, se agrupan y suman todos los asistencias que corresponden a un mismo jugador en un mismo partido 
# (ya que un jugador pudo haber salido y vuelto a entrar, por ejemplo).
#
# Inmediatamente despues, convertimos todos esos registros en tuplas de formato: 
# (id_jugador, cantidad_asistencias_total_en_partido)

rdd_po = rdd_po.reduceByKey(lambda x,y: x+y).map(lambda x: (x[0][0], x[1]))

In [ ]:
# Ahora, nos quedamos con el maximo de asistencias de cada jugador en un partido de playoffs (si el maximo no
# supera el promedio, entonces ese jugador no cumple).
#
# Luego, se toma eso y se convierte en tuplas de formato 
# (id_jugador, [cantidad_asistencias_maximo])

rdd_po = rdd_po.reduceByKey(lambda x,y: max(x,y)).map(lambda x: (x[0], [x[1]])

In [ ]:
# Joineamos por id_jugador ambos RDD, por lo que nos quedarian tuplas de formato:
# (id_jugador, ([cantidad_asistencias_maximo], [nombre, promedio_asistencias]))
#
# Pasa a convertir el segundo elemento de cada registro en una unica lista concatenada de forma:
# [cantidad_asistencias_maximo, nombre, promedio_asistencias]

rdd_combinado = rdd_po.join(rdd_tr).map(lambda x: (x[0], x[1][0] + x[1][1])

In [ ]:
# A lo ultimo, filtramos y nos quedamos solo con aquellos que su cantidad_asistencias_maximo es mayor a su
# promedio_asistencias.
# Paso siguiente, de aquellos jugadores que cumplieron con lo buscado, nos quedamos solo con su nombre.
# Devolvemos la lista de nombres de jugadores que superaron su promedio de asistencias en algun partido de playoffs.

rdd_combinado = rdd_combinado.filter(lambda x: x[1][0] > x[1][2]).map(lambda x: x[1][1])
rdd_combinado.collect()